In [1]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
from io import StringIO

#pd.set_option('max_rows', 20)
#Fuente http://joseluisfernandez.me/movie-recommender/

In [2]:
# Cargamos el archivo de película. movie_genre tiene los géneros de la película separados por "::"
movies_df = pd.read_table('movies.dat', header=None, sep='::', engine='python', names=['movie_id', 'movie_title', 'movie_genre'], encoding = "ISO-8859-1")
print('Movies_df Shape:',movies_df.shape)
#la funcion head se usa para obtener numero de fila. devuelve las primeras n filas del objeto segun la posicion
movies_df.head()

Movies_df Shape: (3883, 3)


movie_id                         movie_title                   movie_genre
0         1                    Toy Story (1995)   Animation|Children's|Comedy
1         2                      Jumanji (1995)  Adventure|Children's|Fantasy
2         3             Grumpier Old Men (1995)                Comedy|Romance
3         4            Waiting to Exhale (1995)                  Comedy|Drama
4         5  Father of the Bride Part II (1995)                        Comedy

In [3]:
#dummies Convierta la variable categórica en variables ficticias / indicadoras.
movies_df.movie_genre.str.get_dummies(sep='|')

Action  Adventure  Animation  Children's  Comedy  Crime  Documentary  \
0          0          0          1           1       1      0            0   
1          0          1          0           1       0      0            0   
2          0          0          0           0       1      0            0   
3          0          0          0           0       1      0            0   
4          0          0          0           0       1      0            0   
...      ...        ...        ...         ...     ...    ...          ...   
3878       0          0          0           0       1      0            0   
3879       0          0          0           0       0      0            0   
3880       0          0          0           0       0      0            0   
3881       0          0          0           0       0      0            0   
3882       0          0          0           0       0      0            0   

      Drama  Fantasy  Film-Noir  Horror  Musical  Mystery  Romance  Sci-Fi  \
0         0        0          0       0        0        0        0       0   
1         0        1          0       0        0        0        0       0   
2         0        0          0       0        0        0        1       0   
3         1        0          0       0        0        0        0       0   
4         0        0          0       0        0        0        0       0   
...     ...      ...        ...     ...      ...      ...      ...     ...   
3878      0        0          0       0        0        0        0       0   
3879      1        0          0       0        0        0        0       0   
3880      1        0          0       0        0        0        0       0   
3881      1        0          0       0        0        0        0       0   
3882      1        0          0       0        0        0        0       0   

      Thriller  War  Western  
0            0    0        0  
1            0    0        0  
2            0    0        0  
3            0    0        0  
4            0    0        0  
...        ...  ...      ...  
3878         0    0        0  
3879         0    0        0  
3880         0    0        0  
3881         0    0        0  
3882         1    0        0  

[3883 rows x 18 columns]

In [4]:
movies_df = pd.concat([movies_df, movies_df.movie_genre.str.get_dummies(sep='|')], axis=1)
movies_df.head()

movie_id                         movie_title                   movie_genre  \
0         1                    Toy Story (1995)   Animation|Children's|Comedy   
1         2                      Jumanji (1995)  Adventure|Children's|Fantasy   
2         3             Grumpier Old Men (1995)                Comedy|Romance   
3         4            Waiting to Exhale (1995)                  Comedy|Drama   
4         5  Father of the Bride Part II (1995)                        Comedy   

   Action  Adventure  Animation  Children's  Comedy  Crime  Documentary  ...  \
0       0          0          1           1       1      0            0  ...   
1       0          1          0           1       0      0            0  ...   
2       0          0          0           0       1      0            0  ...   
3       0          0          0           0       1      0            0  ...   
4       0          0          0           0       1      0            0  ...   

   Fantasy  Film-Noir  Horror  Musical  Mystery  Romance  Sci-Fi  Thriller  \
0        0          0       0        0        0        0       0         0   
1        1          0       0        0        0        0       0         0   
2        0          0       0        0        0        1       0         0   
3        0          0       0        0        0        0       0         0   
4        0          0       0        0        0        0       0         0   

   War  Western  
0    0        0  
1    0        0  
2    0        0  
3    0        0  
4    0        0  

[5 rows x 21 columns]

In [5]:
# la varibles son creados con las categorias
movie_categories = movies_df.columns[3:]
movie_categories

Index(['Action', 'Adventure', 'Animation', 'Children's', 'Comedy', 'Crime',
       'Documentary', 'Drama', 'Fantasy', 'Film-Noir', 'Horror', 'Musical',
       'Mystery', 'Romance', 'Sci-Fi', 'Thriller', 'War', 'Western'],
      dtype='object')

In [6]:
# cargar el archivo de desdcciones, es un conjunto de usuarios con las películas que han visto y su puntuación
# esta igualando lA funcion ratings.dat a ratings 
#pd.read_table importador de datos
ratings = pd.read_table('ratings.dat', header=None, sep='::', engine='python', names=['user_id', 'movie_id', 'rating', 'timestamp'])

# Eliminamos la fecha en que se creó la calificación
del ratings ['timestamp']
ratings.head()


user_id  movie_id  rating
0        1      1193       5
1        1       661       3
2        1       914       3
3        1      3408       4
4        1      2355       5

In [7]:
print('ratings Shape:',ratings.shape)


ratings Shape: (1000209, 3)


In [8]:
#Añadimos el título de la película
#pd.merge Fusionar objetos DataFrame o Series con nombre con una combinación de estilo de base de datos.
ratings = pd.merge(ratings, movies_df, on='movie_id')[['user_id', 'movie_title', 'movie_id','rating']]
print('ratings merge Shape:',ratings.shape)
ratings.head()

ratings merge Shape: (1000209, 4)


user_id                             movie_title  movie_id  rating
0        1  One Flew Over the Cuckoo's Nest (1975)      1193       5
1        2  One Flew Over the Cuckoo's Nest (1975)      1193       5
2       12  One Flew Over the Cuckoo's Nest (1975)      1193       4
3       15  One Flew Over the Cuckoo's Nest (1975)      1193       4
4       17  One Flew Over the Cuckoo's Nest (1975)      1193       5

In [9]:
# Creamos una nueva matriz con las proporciones de cada usuario para todas las películas
#pivot_table crea tabla dinamica como una data frame 
ratings_matriz = ratings.pivot_table(values='rating', index='user_id', columns='movie_title')
print('ratings_matriz Shape:', ratings_matriz.shape)
ratings_matriz.head()

ratings_matriz Shape: (6040, 3706)


movie_title  $1,000,000 Duck (1971)  'Night Mother (1986)  \
user_id                                                     
1                               NaN                   NaN   
2                               NaN                   NaN   
3                               NaN                   NaN   
4                               NaN                   NaN   
5                               NaN                   NaN   

movie_title  'Til There Was You (1997)  'burbs, The (1989)  \
user_id                                                      
1                                  NaN                 NaN   
2                                  NaN                 NaN   
3                                  NaN                 NaN   
4                                  NaN                 NaN   
5                                  NaN                 NaN   

movie_title  ...And Justice for All (1979)  1-900 (1994)  \
user_id                                                    
1                                      NaN           NaN   
2                                      NaN           NaN   
3                                      NaN           NaN   
4                                      NaN           NaN   
5                                      NaN           NaN   

movie_title  10 Things I Hate About You (1999)  101 Dalmatians (1961)  \
user_id                                                                 
1                                          NaN                    NaN   
2                                          NaN                    NaN   
3                                          NaN                    NaN   
4                                          NaN                    NaN   
5                                          NaN                    NaN   

movie_title  101 Dalmatians (1996)  12 Angry Men (1957)  ...  \
user_id                                                  ...   
1                              NaN                  NaN  ...   
2                              NaN                  NaN  ...   
3                              NaN                  NaN  ...   
4                              NaN                  NaN  ...   
5                              NaN                  NaN  ...   

movie_title  Young Poisoner's Handbook, The (1995)  \
user_id                                              
1                                              NaN   
2                                              NaN   
3                                              NaN   
4                                              NaN   
5                                              NaN   

movie_title  Young Sherlock Holmes (1985)  Young and Innocent (1937)  \
user_id                                                                
1                                     NaN                        NaN   
2                                     NaN                        NaN   
3                                     NaN                        NaN   
4                                     NaN                        NaN   
5                                     NaN                        NaN   

movie_title  Your Friends and Neighbors (1998)  Zachariah (1971)  \
user_id                                                            
1                                          NaN               NaN   
2                                          NaN               NaN   
3                                          NaN               NaN   
4                                          NaN               NaN   
5                                          NaN               NaN   

movie_title  Zed & Two Noughts, A (1985)  Zero Effect (1998)  \
user_id                                                        
1                                    NaN                 NaN   
2                                    NaN                 NaN   
3                                    NaN                 NaN   
4                                    NaN                 NaN   
5         

In [10]:
# We fill with mean in the Nan values#
#ratings_matriz.fillna(0, inplace=True)
# .apply se usa para aplicar a una funcion dada a un dataframe completo,(lambda x:x.fillna(x.mean()), axis=0 ) remplaza los NAN con valores numericos
ratings_matriz=ratings_matriz.apply(lambda x:x.fillna(x.mean()), axis=0 )
ratings_matriz=ratings_matriz.apply(lambda x:x - x.mean(), axis=1 )
ratings_matriz.head()

movie_title  $1,000,000 Duck (1971)  'Night Mother (1986)  \
user_id                                                     
1                         -0.215674              0.128728   
2                         -0.210064              0.134337   
3                         -0.213476              0.130926   
4                         -0.212675              0.131726   
5                         -0.182296              0.162105   

movie_title  'Til There Was You (1997)  'burbs, The (1989)  \
user_id                                                      
1                            -0.550393           -0.331810   
2                            -0.544784           -0.326200   
3                            -0.548195           -0.329612   
4                            -0.547394           -0.328811   
5                            -0.517016           -0.298432   

movie_title  ...And Justice for All (1979)  1-900 (1994)  \
user_id                                                    
1                                 0.470867     -0.742701   
2                                 0.476476     -0.737091   
3                                 0.473065     -0.740503   
4                                 0.473866     -0.739702   
5                                 0.504244     -0.709323   

movie_title  10 Things I Hate About You (1999)  101 Dalmatians (1961)  \
user_id                                                                 
1                                     0.180156               0.353760   
2                                     0.185766               0.359369   
3                                     0.182354               0.355957   
4                                     0.183155               0.356758   
5                                     0.213534               0.387137   

movie_title  101 Dalmatians (1996)  12 Angry Men (1957)  ...  \
user_id                                                  ...   
1                        -0.195997             1.052754  ...   
2                        -0.190388             1.058363  ...   
3                        -0.193799             1.054952  ...   
4                        -0.192999             1.055752  ...   
5                        -0.162620             1.086131  ...   

movie_title  Young Poisoner's Handbook, The (1995)  \
user_id                                              
1                                         0.390211   
2                                         0.395820   
3                                         0.392409   
4                                         0.393209   
5                                         0.423588   

movie_title  Young Sherlock Holmes (1985)  Young and Innocent (1937)  \
user_id                                                                
1                                0.147801                   0.057299   
2                                0.153410                   0.062909   
3                                0.149999                   0.059497   
4                                0.150799                   0.060298   
5                                0.181178                   0.090677   

movie_title  Your Friends and Neighbors (1998)  Zachariah (1971)  \
user_id                                                            
1                                     0.133446          0.257299   
2                                     0.139055          0.262909   
3                                     0.135644          0.259497   
4                                     0.136445          0.260298   
5                                     0.166823          0.290677   

movie_title  Zed & Two Noughts, A (1985)  Zero Effect (1998)  \
user_id                                                        
1                               0.171092            0.508130   
2                               0.176702            0.513739   
3                               0.173290            0.510328   
4                               0.174091            0.511128   
5         

In [11]:

ratings_matriz


movie_title  $1,000,000 Duck (1971)  'Night Mother (1986)  \
user_id                                                     
1                         -0.215674              0.128728   
2                         -0.210064              0.134337   
3                         -0.213476              0.130926   
4                         -0.212675              0.131726   
5                         -0.182296              0.162105   
...                             ...                   ...   
6036                      -0.143992             -0.171019   
6037                      -0.197548              0.146853   
6038                      -0.210879              0.133523   
6039                      -0.209104              0.135298   
6040                      -0.184900              0.159501   

movie_title  'Til There Was You (1997)  'burbs, The (1989)  \
user_id                                                      
1                            -0.550393           -0.331810   
2                            -0.544784           -0.326200   
3                            -0.548195           -0.329612   
4                            -0.547394           -0.328811   
5                            -0.517016           -0.298432   
...                                ...                 ...   
6036                         -0.478711           -0.260128   
6037                         -0.532268           -0.313684   
6038                         -0.545598           -0.327015   
6039                         -0.543823           -0.325240   
6040                         -0.519620           -0.301036   

movie_title  ...And Justice for All (1979)  1-900 (1994)  \
user_id                                                    
1                                 0.470867     -0.742701   
2                                 0.476476     -0.737091   
3                                 0.473065     -0.740503   
4                                 0.473866     -0.739702   
5                                 0.504244     -0.709323   
...                                    ...           ...   
6036                              0.542549     -0.671019   
6037                              0.488992     -0.724576   
6038                              0.475662     -0.737906   
6039                              0.477437     -0.736131   
6040                              0.501641     -0.711927   

movie_title  10 Things I Hate About You (1999)  101 Dalmatians (1961)  \
user_id                                                                 
1                                     0.180156               0.353760   
2                                     0.185766               0.359369   
3                                     0.182354               0.355957   
4                                     0.183155               0.356758   
5                                     0.213534               0.387137   
...                                        ...                    ...   
6036                                 -1.171019               0.828981   
6037                                  0.198282               0.371885   
6038                                  0.184951               0.358554   
6039                                  0.186726               0.360329   
6040                                  0.210930               0.384533   

movie_title  101 Dalmatians (1996)  12 Angry Men (1957)  ...  \
user_id                                                  ...   
1                        -0.195997             1.052754  ...   
2                        -0.190388             1.058363  ...   
3                        -0.193799             1.054952  ...   
4                        -0.192999             1.055752  ...   
5                        -0.162620             1.086131  ...   
...                            ...                  ...  ...   
6036                     -0.124316             1.124436  ...   
6037                     -0.177872             0.775424  ...   
6038                     -0.

In [12]:
movie_index = ratings_matriz.columns
print('movie_index Shape:', movie_index.shape)
movie_index

movie_index Shape: (3706,)


Index(['$1,000,000 Duck (1971)', ''Night Mother (1986)',
       ''Til There Was You (1997)', ''burbs, The (1989)',
       '...And Justice for All (1979)', '1-900 (1994)',
       '10 Things I Hate About You (1999)', '101 Dalmatians (1961)',
       '101 Dalmatians (1996)', '12 Angry Men (1957)',
       ...
       'Young Poisoner's Handbook, The (1995)', 'Young Sherlock Holmes (1985)',
       'Young and Innocent (1937)', 'Your Friends and Neighbors (1998)',
       'Zachariah (1971)', 'Zed & Two Noughts, A (1985)', 'Zero Effect (1998)',
       'Zero Kelvin (Kjærlighetens kjøtere) (1995)', 'Zeus and Roxanne (1997)',
       'eXistenZ (1999)'],
      dtype='object', name='movie_title', length=3706)

In [13]:
#the position in Toy Story's movie listing
list(movie_index).index('Toy Story (1995)')

3411

In [14]:
#df.shape visualizar numero de filas y columnas
print('ratings_matriz Shape:', ratings_matriz.shape)
print('ratings_matriz.T Shape:', ratings_matriz.T.shape)
ratings_matriz.T

ratings_matriz Shape: (6040, 3706)
ratings_matriz.T Shape: (3706, 6040)


user_id                                         1         2         3     \
movie_title                                                                
$1,000,000 Duck (1971)                     -0.215674 -0.210064 -0.213476   
'Night Mother (1986)                        0.128728  0.134337  0.130926   
'Til There Was You (1997)                  -0.550393 -0.544784 -0.548195   
'burbs, The (1989)                         -0.331810 -0.326200 -0.329612   
...And Justice for All (1979)               0.470867  0.476476  0.473065   
...                                              ...       ...       ...   
Zed & Two Noughts, A (1985)                 0.171092  0.176702  0.173290   
Zero Effect (1998)                          0.508130  0.513739  0.510328   
Zero Kelvin (Kjærlighetens kjøtere) (1995)  0.257299  0.262909  0.259497   
Zeus and Roxanne (1997)                    -0.720962 -0.715352 -0.718764   
eXistenZ (1999)                             0.013397  0.019006  0.015595   

user_id                                         4         5         6     \
movie_title                                                                
$1,000,000 Duck (1971)                     -0.212675 -0.182296 -0.217548   
'Night Mother (1986)                        0.131726  0.162105  0.126854   
'Til There Was You (1997)                  -0.547394 -0.517016 -0.552267   
'burbs, The (1989)                         -0.328811 -0.298432 -0.333684   
...And Justice for All (1979)               0.473866  0.504244  0.468993   
...                                              ...       ...       ...   
Zed & Two Noughts, A (1985)                 0.174091  0.204470  0.169218   
Zero Effect (1998)                          0.511128  0.541507  0.506256   
Zero Kelvin (Kjærlighetens kjøtere) (1995)  0.260298  0.290677  0.255425   
Zeus and Roxanne (1997)                    -0.717963 -0.687584 -0.722836   
eXistenZ (1999)                             0.016395  0.046774  0.011523   

user_id                                         7         8         9     \
movie_title                                                                
$1,000,000 Duck (1971)                     -0.216115 -0.221737 -0.208101   
'Night Mother (1986)                        0.128286  0.122664  0.136300   
'Til There Was You (1997)                  -0.550835 -0.556457 -0.542821   
'burbs, The (1989)                         -0.332251 -0.337873 -0.324237   
...And Justice for All (1979)               0.470426  0.464803  0.478439   
...                                              ...       ...       ...   
Zed & Two Noughts, A (1985)                 0.170651  0.165029  0.178665   
Zero Effect (1998)                          0.507688  0.502066  0.515702   
Zero Kelvin (Kjærlighetens kjøtere) (1995)  0.256858  0.251236  0.264872   
Zeus and Roxanne (1997)                    -0.721403 -0.727025 -0.713389   
eXistenZ (1999)                             0.012955 -1.248764  0.020969   

user_id                                         10    ...      6031      6032  \
movie_title                                           ...                       
$1,000,000 Duck (1971)                     -0.262994  ... -0.210600 -0.217917   
'Night Mother (1986)                        0.081407  ...  0.133802  0.126485   
'Til There Was You (1997)                  -0.597714  ... -0.545319 -0.552636   
'burbs, The (1989)                          0.709979  ... -0.326736 -0.334053   
...And Justice for All (1979)               0.423546  ...  0.475941  0.468624   
...                                              ...  ...       ...       ...   
Zed & Two Noughts, A (1985)                 0.123772  ...  0.176166  0.168849   
Zero Effect (1998)                          0.460809  ...  0.513204  0.505887   
Zero Kelvin (Kjærlighetens kjøtere) (1995)  0.209979  ...  0.262373  0.255056   
Zeus and Roxanne (1997)                    -0.768282  ... -0.715888 -0.723205   
eXistenZ (1999)                            -0.033924  ...  0.

In [15]:
#np.corrcoef(ratings_matriz.T)

In [16]:
corr_matrix = np.corrcoef(ratings_matriz.T)
print('corr_matrix Shape:', corr_matrix.shape)

corr_matrix Shape: (3706, 3706)


In [17]:
corr_matrix

array([[ 1.00000000e+00,  3.38790195e-02,  5.68837908e-02, ...,
         2.66382718e-01,  1.05257034e-01, -2.48429774e-02],
       [ 3.38790195e-02,  1.00000000e+00, -1.32194687e-02, ...,
         1.00183847e-01,  3.26130706e-02, -1.57568962e-02],
       [ 5.68837908e-02, -1.32194687e-02,  1.00000000e+00, ...,
         1.92041086e-01,  7.85162956e-02,  1.65494058e-04],
       ...,
       [ 2.66382718e-01,  1.00183847e-01,  1.92041086e-01, ...,
         1.00000000e+00,  3.15144002e-01, -3.21763262e-02],
       [ 1.05257034e-01,  3.26130706e-02,  7.85162956e-02, ...,
         3.15144002e-01,  1.00000000e+00, -1.02228164e-02],
       [-2.48429774e-02, -1.57568962e-02,  1.65494058e-04, ...,
        -3.21763262e-02, -1.02228164e-02,  1.00000000e+00]])

In [18]:
movies_df['movie_title'][0]

'Toy Story (1995)'

In [19]:

favorit_movie = 'Toy Story (1995)'

favorita_movie_index = list(movie_index).index(favorit_movie) #the position in favorit_movie movie listing

P = corr_matrix[favorita_movie_index]

list(movie_index[(P>0.4) & (P<1.0)])

[]

In [20]:
list(movie_index[(P>0.2) & (P<1.0)])

['Aladdin (1992)', "Bug's Life, A (1998)", 'Toy Story 2 (1999)']

In [21]:
favorita_movie_index

3411

In [22]:
#The Toy Story film's correlation list with each user
print('P Shape:', P.shape)
P

P Shape: (3706,)


array([-0.06290601, -0.01869119, -0.04195901, ..., -0.15931814,
       -0.02565778, -0.00534232])

In [23]:
#A function that returns the correlation vector for a movie
#np.corrcoef función que devuelve una matriz de correlaciones
def get_similar_movie(movie):
    corr_matrix = np.corrcoef(ratings_matriz.T)
    movie_idx = list(movie_index).index(movie)
    return corr_matrix[movie_idx]

#We return movies that are more similar to the tastes of a model user.
#If we want to recommend movies to a user, we get the list of movies they've watched and add up the correlations
#of those movies with all the others to return the movies with a greater total correlation..
def get_movie_recomendations(user):
    corr_matrix = np.corrcoef(ratings_matriz.T)
    similar_movies = np.zeros(corr_matrix.shape[0])
    for movie_id in user:
        similar_movies = similar_movies + get_similar_movie(movie_id)
        similars_df = pd.DataFrame({
        'Titulo': movie_index,
        'sum_similar': similar_movies
        })
    similars_df = similars_df[~(similars_df.Titulo.isin(user))]
    similars_df = similars_df.sort_values(by=['sum_similar'], ascending=False)
    return similars_df

In [24]:
#We're going to recommend movies to user 21, who has seen the following movies with their corresponding rating
user_21 = 21
ratings[ratings.user_id==user_21].sort_values(by=['rating'], ascending=False)

user_id                                        movie_title  movie_id  \
583304       21                                  Titan A.E. (2000)      3745   
707307       21      Princess Mononoke, The (Mononoke Hime) (1997)      3000   
70742        21  Star Wars: Episode VI - Return of the Jedi (1983)      1210   
239644       21        South Park: Bigger, Longer and Uncut (1999)      2700   
487530       21                  Mad Max Beyond Thunderdome (1985)      3704   
707652       21      Little Nemo: Adventures in Slumberland (1992)      2800   
708015       21                  Stop! Or My Mom Will Shoot (1992)      3268   
706889       21                      Brady Bunch Movie, The (1995)       585   
623947       21                             Iron Giant, The (1999)      2761   
619784       21                              Wild Wild West (1999)      2701   
4211         21                               Bug's Life, A (1998)      2355   
368056       21                                       Akira (1988)      1274   
226126       21                    Who Framed Roger Rabbit? (1988)      2987   
41633        21                                   Toy Story (1995)         1   
34978        21                                     Aladdin (1992)       588   
33432        21                                        Antz (1998)      2294   
18917        21                                       Bambi (1942)      2018   
612215       21                       Devil's Advocate, The (1997)      1645   
617656       21                        Prince of Egypt, The (1998)      2394   
440983       21                                   Pinocchio (1940)       596   
707674       21    Messenger: The Story of Joan of Arc, The (1999)      3053   
708194       21                               House Party 2 (1991)      3774   

        rating  
583304       5  
707307       5  
70742        5  
239644       5  
487530       4  
707652       4  
708015       3  
706889       3  
623947       3  
619784       3  
4211         3  
368056       3  
226126       3  
41633        3  
34978        3  
33432        3  
18917        1  
612215       1  
617656       1  
440983       1  
707674       1  
708194       1

In [25]:

#Generamos un listado de títulos de películas del usuario 21 para poder añadirlo a las funciones.
#The df ratings is taken and the column movie_title.
user_film_21_list = ratings[ratings.user_id==user_21].movie_title.tolist()
user_film_21_list

["Bug's Life, A (1998)",
 'Bambi (1942)',
 'Antz (1998)',
 'Aladdin (1992)',
 'Toy Story (1995)',
 'Star Wars: Episode VI - Return of the Jedi (1983)',
 'Who Framed Roger Rabbit? (1988)',
 'South Park: Bigger, Longer and Uncut (1999)',
 'Akira (1988)',
 'Pinocchio (1940)',
 'Mad Max Beyond Thunderdome (1985)',
 'Titan A.E. (2000)',
 "Devil's Advocate, The (1997)",
 'Prince of Egypt, The (1998)',
 'Wild Wild West (1999)',
 'Iron Giant, The (1999)',
 'Brady Bunch Movie, The (1995)',
 'Princess Mononoke, The (Mononoke Hime) (1997)',
 'Little Nemo: Adventures in Slumberland (1992)',
 'Messenger: The Story of Joan of Arc, The (1999)',
 'Stop! Or My Mom Will Shoot (1992)',
 'House Party 2 (1991)']

In [26]:

recomendations_21_user = get_movie_recomendations(user_film_21_list)
recomendations_21_user.Titulo.head(20)

3055    Snow White and the Seven Dwarfs (1937)
1865                 Lady and the Tramp (1955)
679                          Cinderella (1950)
1002                              Dumbo (1941)
1939                     Lion King, The (1994)
324                Beauty and the Beast (1991)
7                        101 Dalmatians (1961)
2770           Rescuers Down Under, The (1990)
1948                Little Mermaid, The (1989)
3412                        Toy Story 2 (1999)
2808                         Robin Hood (1973)
3026                    Sleeping Beauty (1959)
1739          James and the Giant Peach (1996)
3275                             Tarzan (1999)
97                  Alice in Wonderland (1951)
1781                   Jungle Book, The (1967)
1611       Hunchback of Notre Dame, The (1996)
2771                      Rescuers, The (1977)
2432                   Oliver & Company (1988)
1111                           Fantasia (1940)
Name: Titulo, dtype: object